In [1]:
import sympy
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# default_pysr_params = dict(
#     populations=30,
#     model_selection="best",
# )

In [3]:
# model = PySRRegressor(
#     niterations=30,
#     binary_operators=["+", "*", "-", "/"],
#     unary_operators=["cos", "exp", "sin"],
#     batching=True,
#     batch_size=64,
#     **default_pysr_params,
# )

In [4]:
data = pd.read_csv("../data/train.csv")

In [5]:
X = data.drop(columns=["critical_temp"])
y = data["critical_temp"]

X_test, X_train, y_test, y_train = train_test_split(
        X, y, test_size=0.9, random_state=1702
    )
y_test = np.clip(y_test, 1e-6, None)
y_train = np.clip(y_train, 1e-6, None)

In [6]:
# model.fit(X_train, y_train)

In [7]:
from sklearn.cluster import KMeans
from script_sr import Clustering_SR, cross_validation

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [8]:
cluster = KMeans()
model = Clustering_SR(clusterer=cluster, 
              n_iterations=10, 
              maxsize=50, 
              maxdepth=10,
              binary_operators=["+", "*"],
              unary_operators=["exp"],
              select_k_features=10)

In [9]:
model.fit(X_train, y_train)

c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:1469: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Using features ['wtd_mean_fie' 'wtd_gmean_fie' 'wtd_mean_atomic_radius'
 'wtd_mean_Density' 'gmean_Density' 'range_FusionHeat' 'std_FusionHeat'
 'wtd_mean_ThermalConductivity' 'wtd_gmean_ThermalConductivity'
 'std_ThermalConductivity']
Compiling Julia backend...
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.743e+01  0.000e+00  y = 4.7056
3           1.689e+01  1.578e-02  y = range_FusionHeat + 5.3661
5           1.654e+01  1.055e-02  y = (wtd_mean_ThermalConductivity * -0.73924) + 4.2015
7           1.619e+01  1.057e-02  y = (wtd_gmean_ThermalConductivity * -1.063) + (range_Fusi...
                                      onHeat + 6.0141)
9           1.580e+01  1.229e-02  y = range_FusionHeat + ((range_FusionHeat + 6.3373) + (wtd...
                                      _mean_ThermalConductivity * -0.89111))
11          1.552e+01  8.952e-03  y = ((range_FusionHeat + (range_FusionHeat + 5.8131)) + 0....
                                      085198) + (wtd_mean_ThermalConductivity * -0.89111)
13          1.512e+01  1.299e-02  y = exp((std_ThermalConductivity * (exp(wtd_mean_fie) + -0...
                                      .97319)) + (ran

c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:1469: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Using features ['std_atomic_mass' 'wtd_std_atomic_mass' 'wtd_std_fie' 'std_Density'
 'mean_FusionHeat' 'wtd_gmean_ThermalConductivity'
 'wtd_entropy_ThermalConductivity' 'wtd_std_ThermalConductivity'
 'wtd_mean_Valence' 'wtd_gmean_Valence']
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:1469: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:281

───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           9.556e+02  0.000e+00  y = 56.301
3           9.299e+02  1.358e-02  y = wtd_std_ThermalConductivity * 64.764
4           8.727e+02  6.341e-02  y = exp(wtd_std_ThermalConductivity + 3.0514)
6           8.495e+02  1.345e-02  y = exp(wtd_std_ThermalConductivity + 2.7474) * 1.4883
7           7.340e+02  1.461e-01  y = exp(exp(wtd_mean_Valence * -0.95931)) * 6.1925
10          7.336e+02  1.748e-04  y = ((exp(wtd_gmean_Valence) * -439.31) + (mean_FusionHeat...
                                       * 6.4839)) + 266.06
12          7.260e+02  5.205e-03  y = ((exp(wtd_gmean_Valence) * -439.31) + ((mean_FusionHea...
                                      t + mean_FusionHeat) * 6.4839)) + 266.06
13          7.197e+02  8.585e-03  y = (((wtd_gmean_Valence * (wtd_mean_Valence + -20.603)) +...
                                       -14.301) * (wtd_me

Using features ['wtd_mean_atomic_mass' 'mean_fie' 'std_atomic_radius' 'wtd_gmean_Density'
 'mean_ElectronAffinity' 'wtd_mean_ElectronAffinity'
 'wtd_gmean_ElectronAffinity' 'wtd_entropy_FusionHeat'
 'mean_ThermalConductivity' 'gmean_ThermalConductivity']
[ Info: Started!


...

  - outputs\20250715_111305_HnhRaJ\hall_of_fame.csv


[ Info: Final population:
[ Info: Results saved to:
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:1469: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Using features ['wtd_mean_atomic_mass' 'wtd_mean_fie' 'entropy_fie'
 'wtd_entropy_atomic_radius' 'wtd_range_ElectronAffinity'
 'wtd_std_FusionHeat' 'wtd_mean_Valence' 'gmean_Valence'
 'wtd_gmean_Valence' 'wtd_range_Valence']
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:1469: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\julia24002\.conda\envs\glm_

,clusterer,KMeans()
,n_iterations,10
,maxsize,50
,maxdepth,10
,binary_operators,"['+', '*']"
,unary_operators,['exp']
,select_k_features,10
,n_clusters,8
,init,'k-means++'
,n_init,'auto'
,max_iter,300


Error in callback _flush_stdio (for post_execute), with arguments args (),kwargs {}:


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

In [10]:
model.predict(X_test)

c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


array([60.81063025, 63.46823403, 66.33427496, ...,  6.43650905,
       39.94489703, 66.33427496])

Error in callback _flush_stdio (for post_execute), with arguments args (),kwargs {}:


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 4095: unexpected end of data

In [ ]:
cross_validation(X = X_train, 
                y = y_train,
                clusterer=cluster, 
                n_iterations=10, 
                maxsize=50, 
                maxdepth=10,
                binary_operators=["+", "*"],
                unary_operators=["exp"],
                select_k_features=10)

c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:1469: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Using features ['mean_atomic_mass' 'wtd_mean_atomic_mass' 'gmean_atomic_mass'
 'entropy_fie' 'wtd_mean_atomic_radius' 'wtd_std_atomic_radius'
 'wtd_mean_ElectronAffinity' 'range_FusionHeat' 'wtd_std_FusionHeat'
 'wtd_mean_Valence']
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:1469: UserWarning: Note: Using a large maxsize for the equation search will be exponentially slower and use significant memory.
  warnings.warn(
c:\Users\julia24002\.conda\envs\glm_gam\lib\site-packages\pysr\sr.py:2811: UserWa